In [1]:
#import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#import data
train=pd.read_csv('train.csv',index_col='id')
test=pd.read_csv('test.csv',index_col='id')
#test_df=test.iloc[:,0:7]

test['num_orders']=-999 #default value
df=pd.concat((train,test),axis=0)

In [3]:
df.head(15)

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
id,,,,,,,,
1379560,1,55,1885,136.83,152.29,0,0,177
1466964,1,55,1993,136.83,135.83,0,0,270
1346989,1,55,2539,134.86,135.86,0,0,189
1338232,1,55,2139,339.50,437.53,0,0,54
1448490,1,55,2631,243.50,242.50,0,0,40
1270037,1,55,1248,251.23,252.23,0,0,28
1191377,1,55,1778,183.36,184.36,0,0,190
1499955,1,55,1062,182.36,183.36,0,0,391
1025244,1,55,2707,193.06,192.06,0,0,472


In [4]:
#categorical features
categorical_features_pos=[0,1,2]

In [5]:
#addition of feature as tax
df['tax']=((df['checkout_price']-df['base_price'])/df['base_price'])*100

In [6]:
#keep num)orders at last
cols_at_end = ['num_orders']
df = df[[c for c in df if c not in cols_at_end] + [c for c in cols_at_end if c in df]]

In [7]:
#train data
train_processed=df.loc[df.num_orders!=-999]
#test data
columns=[column for column in df.columns if column!='num_orders']
test_processed=df.loc[df.num_orders==-999,columns]

In [8]:
#all columns of train data
train_processed.columns

Index(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'tax', 'num_orders'],
      dtype='object')

In [9]:
#dependent and independent columns
X=train_processed.loc[:,:'tax'].values.astype('float')
y=train_processed['num_orders'].ravel()

In [10]:
#test data for which prediction is to b emade
test_processed=test_processed.loc[:,:'tax'].values.astype('float')

In [11]:
#shape of X and y
print(X.shape)
print(y.shape)

(456548, 8)
(456548,)


In [12]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(365238, 8) (365238,)
(91310, 8) (91310,)


In [13]:
#Normalization

In [13]:
from catboost import CatBoostRegressor

In [14]:
cb_model = CatBoostRegressor()

In [15]:
from sklearn.model_selection import GridSearchCV
parameters = {'depth':[5,10],
          'iterations':[500,1000],
          'learning_rate':[0.001,0.01,0.1], 
          'l2_leaf_reg':[1,3,5],
          'border_count':[40],
          'thread_count':[4]}

clf=GridSearchCV(cb_model,param_grid=parameters,cv=2)

In [16]:
# Fit the grid search to the data
clf.fit(X_train,y_train)

0:	learn: 480.1159841	total: 134ms	remaining: 1m 6s
1:	learn: 479.8597354	total: 189ms	remaining: 47s
2:	learn: 479.6137759	total: 243ms	remaining: 40.2s
3:	learn: 479.3575230	total: 304ms	remaining: 37.6s
4:	learn: 479.1016448	total: 362ms	remaining: 35.8s
5:	learn: 478.8464512	total: 420ms	remaining: 34.5s
6:	learn: 478.5913202	total: 479ms	remaining: 33.7s
7:	learn: 478.3450716	total: 531ms	remaining: 32.7s
8:	learn: 478.1011420	total: 587ms	remaining: 32s
9:	learn: 477.8473726	total: 648ms	remaining: 31.8s
10:	learn: 477.6013693	total: 703ms	remaining: 31.2s
11:	learn: 477.3483113	total: 763ms	remaining: 31s
12:	learn: 477.1057451	total: 851ms	remaining: 31.9s
13:	learn: 476.8692704	total: 905ms	remaining: 31.4s
14:	learn: 476.6169282	total: 957ms	remaining: 31s
15:	learn: 476.3727744	total: 1.01s	remaining: 30.6s
16:	learn: 476.1383504	total: 1.07s	remaining: 30.5s
17:	learn: 475.8930007	total: 1.13s	remaining: 30.3s
18:	learn: 475.6423511	total: 1.2s	remaining: 30.3s
19:	learn: 4

158:	learn: 444.6636683	total: 9.63s	remaining: 20.7s
159:	learn: 444.4578958	total: 9.69s	remaining: 20.6s
160:	learn: 444.2557553	total: 9.74s	remaining: 20.5s
161:	learn: 444.0623017	total: 9.79s	remaining: 20.4s
162:	learn: 443.8574062	total: 9.86s	remaining: 20.4s
163:	learn: 443.6584672	total: 9.92s	remaining: 20.3s
164:	learn: 443.4544909	total: 9.97s	remaining: 20.2s
165:	learn: 443.2552145	total: 10s	remaining: 20.2s
166:	learn: 443.0657028	total: 10.1s	remaining: 20.1s
167:	learn: 442.8657326	total: 10.1s	remaining: 20s
168:	learn: 442.6646853	total: 10.2s	remaining: 20s
169:	learn: 442.4730667	total: 10.3s	remaining: 19.9s
170:	learn: 442.2739763	total: 10.3s	remaining: 19.8s
171:	learn: 442.0842190	total: 10.4s	remaining: 19.8s
172:	learn: 441.8949257	total: 10.4s	remaining: 19.7s
173:	learn: 441.7008184	total: 10.5s	remaining: 19.6s
174:	learn: 441.5012864	total: 10.5s	remaining: 19.5s
175:	learn: 441.3055823	total: 10.6s	remaining: 19.5s
176:	learn: 441.1093670	total: 10.

314:	learn: 416.8007101	total: 19s	remaining: 11.2s
315:	learn: 416.6495507	total: 19.1s	remaining: 11.1s
316:	learn: 416.4853631	total: 19.2s	remaining: 11.1s
317:	learn: 416.3313895	total: 19.2s	remaining: 11s
318:	learn: 416.1712343	total: 19.3s	remaining: 10.9s
319:	learn: 416.0109929	total: 19.3s	remaining: 10.9s
320:	learn: 415.8619996	total: 19.4s	remaining: 10.8s
321:	learn: 415.6996179	total: 19.5s	remaining: 10.8s
322:	learn: 415.5417497	total: 19.5s	remaining: 10.7s
323:	learn: 415.3795528	total: 19.6s	remaining: 10.6s
324:	learn: 415.2270966	total: 19.7s	remaining: 10.6s
325:	learn: 415.0662476	total: 19.7s	remaining: 10.5s
326:	learn: 414.9084351	total: 19.8s	remaining: 10.5s
327:	learn: 414.7480314	total: 19.8s	remaining: 10.4s
328:	learn: 414.5878862	total: 19.9s	remaining: 10.3s
329:	learn: 414.4316009	total: 20s	remaining: 10.3s
330:	learn: 414.2793326	total: 20s	remaining: 10.2s
331:	learn: 414.1309387	total: 20.1s	remaining: 10.2s
332:	learn: 413.9701708	total: 20.2s

470:	learn: 394.7603116	total: 27s	remaining: 1.66s
471:	learn: 394.6335827	total: 27.1s	remaining: 1.6s
472:	learn: 394.5101224	total: 27.1s	remaining: 1.55s
473:	learn: 394.3881105	total: 27.2s	remaining: 1.49s
474:	learn: 394.2622987	total: 27.2s	remaining: 1.43s
475:	learn: 394.1346258	total: 27.3s	remaining: 1.38s
476:	learn: 394.0052078	total: 27.3s	remaining: 1.32s
477:	learn: 393.8780719	total: 27.4s	remaining: 1.26s
478:	learn: 393.7595890	total: 27.4s	remaining: 1.2s
479:	learn: 393.6386762	total: 27.5s	remaining: 1.15s
480:	learn: 393.5184778	total: 27.6s	remaining: 1.09s
481:	learn: 393.3930717	total: 27.6s	remaining: 1.03s
482:	learn: 393.2724595	total: 27.7s	remaining: 974ms
483:	learn: 393.1463304	total: 27.7s	remaining: 917ms
484:	learn: 393.0235725	total: 27.8s	remaining: 860ms
485:	learn: 392.8956509	total: 27.9s	remaining: 802ms
486:	learn: 392.7700126	total: 27.9s	remaining: 745ms
487:	learn: 392.6453252	total: 28s	remaining: 688ms
488:	learn: 392.5169762	total: 28s

127:	learn: 442.6304236	total: 7.43s	remaining: 21.6s
128:	learn: 442.4195684	total: 7.5s	remaining: 21.6s
129:	learn: 442.2231863	total: 7.56s	remaining: 21.5s
130:	learn: 442.0214995	total: 7.61s	remaining: 21.4s
131:	learn: 441.8173981	total: 7.67s	remaining: 21.4s
132:	learn: 441.6248798	total: 7.71s	remaining: 21.3s
133:	learn: 441.4209011	total: 7.76s	remaining: 21.2s
134:	learn: 441.2134840	total: 7.82s	remaining: 21.1s
135:	learn: 441.0054657	total: 7.87s	remaining: 21.1s
136:	learn: 440.8011669	total: 7.92s	remaining: 21s
137:	learn: 440.5925493	total: 7.97s	remaining: 20.9s
138:	learn: 440.3975257	total: 8.02s	remaining: 20.8s
139:	learn: 440.1967837	total: 8.08s	remaining: 20.8s
140:	learn: 439.9989557	total: 8.13s	remaining: 20.7s
141:	learn: 439.7970316	total: 8.17s	remaining: 20.6s
142:	learn: 439.5902585	total: 8.23s	remaining: 20.5s
143:	learn: 439.3897595	total: 8.28s	remaining: 20.5s
144:	learn: 439.1889590	total: 8.34s	remaining: 20.4s
145:	learn: 438.9953844	total: 

...

406:	learn: 288.2998226	total: 24.4s	remaining: 5.58s
407:	learn: 288.2205089	total: 24.5s	remaining: 5.52s
408:	learn: 288.1228893	total: 24.5s	remaining: 5.46s
409:	learn: 288.0622626	total: 24.6s	remaining: 5.39s
410:	learn: 287.9864700	total: 24.6s	remaining: 5.33s
411:	learn: 287.9232482	total: 24.7s	remaining: 5.27s
412:	learn: 287.8428166	total: 24.7s	remaining: 5.21s
413:	learn: 287.7883671	total: 24.8s	remaining: 5.15s
414:	learn: 287.7201055	total: 24.9s	remaining: 5.09s
415:	learn: 287.6261991	total: 24.9s	remaining: 5.03s
416:	learn: 287.5816847	total: 25s	remaining: 4.98s
417:	learn: 287.5296018	total: 25.2s	remaining: 4.93s
418:	learn: 287.4786350	total: 25.2s	remaining: 4.88s
419:	learn: 287.4077611	total: 25.3s	remaining: 4.82s
420:	learn: 287.3501470	total: 25.3s	remaining: 4.76s
421:	learn: 287.2925573	total: 25.4s	remaining: 4.69s
422:	learn: 287.2357724	total: 25.5s	remaining: 4.64s
423:	learn: 287.1885418	total: 25.6s	remaining: 4.59s
424:	learn: 287.1421648	total:

61:	learn: 370.8374753	total: 3.84s	remaining: 27.1s
62:	learn: 369.8450845	total: 3.89s	remaining: 27s
63:	learn: 368.8903975	total: 3.95s	remaining: 26.9s
64:	learn: 367.9258813	total: 4.03s	remaining: 26.9s
65:	learn: 367.0318817	total: 4.15s	remaining: 27.3s
66:	learn: 366.0426206	total: 4.23s	remaining: 27.3s
67:	learn: 365.1348052	total: 4.28s	remaining: 27.2s
68:	learn: 364.2251329	total: 4.34s	remaining: 27.1s
69:	learn: 363.3811052	total: 4.4s	remaining: 27s
70:	learn: 362.4532478	total: 4.45s	remaining: 26.9s
71:	learn: 361.6480861	total: 4.5s	remaining: 26.8s
72:	learn: 360.8118663	total: 4.56s	remaining: 26.7s
73:	learn: 359.9308922	total: 4.62s	remaining: 26.6s
74:	learn: 359.1528247	total: 4.67s	remaining: 26.5s
75:	learn: 358.3645201	total: 4.73s	remaining: 26.4s
76:	learn: 357.5661662	total: 4.79s	remaining: 26.3s
77:	learn: 356.7105722	total: 4.85s	remaining: 26.2s
78:	learn: 355.8998991	total: 4.9s	remaining: 26.1s
79:	learn: 355.1306267	total: 4.95s	remaining: 26s
80

217:	learn: 302.7254751	total: 13.2s	remaining: 17.1s
218:	learn: 302.5047182	total: 13.2s	remaining: 17s
219:	learn: 302.3521060	total: 13.3s	remaining: 16.9s
220:	learn: 302.1946523	total: 13.3s	remaining: 16.8s
221:	learn: 302.0116573	total: 13.4s	remaining: 16.8s
222:	learn: 301.8693469	total: 13.4s	remaining: 16.7s
223:	learn: 301.6816395	total: 13.5s	remaining: 16.6s
224:	learn: 301.5060384	total: 13.6s	remaining: 16.6s
225:	learn: 301.3565234	total: 13.6s	remaining: 16.5s
226:	learn: 301.2105566	total: 13.7s	remaining: 16.4s
227:	learn: 301.0787430	total: 13.7s	remaining: 16.4s
228:	learn: 300.9602055	total: 13.8s	remaining: 16.3s
229:	learn: 300.7347721	total: 13.8s	remaining: 16.2s
230:	learn: 300.5529287	total: 13.9s	remaining: 16.2s
231:	learn: 300.4303992	total: 13.9s	remaining: 16.1s
232:	learn: 300.2714664	total: 14s	remaining: 16s
233:	learn: 300.1183442	total: 14.1s	remaining: 16s
234:	learn: 299.9512987	total: 14.1s	remaining: 15.9s
235:	learn: 299.8279203	total: 14.2s

372:	learn: 284.4917779	total: 21.2s	remaining: 7.22s
373:	learn: 284.4211016	total: 21.3s	remaining: 7.17s
374:	learn: 284.3180824	total: 21.3s	remaining: 7.11s
375:	learn: 284.2494493	total: 21.4s	remaining: 7.05s
376:	learn: 284.1809645	total: 21.4s	remaining: 6.99s
377:	learn: 284.0714343	total: 21.5s	remaining: 6.93s
378:	learn: 283.9783264	total: 21.5s	remaining: 6.87s
379:	learn: 283.9115734	total: 21.6s	remaining: 6.82s
380:	learn: 283.8433212	total: 21.6s	remaining: 6.76s
381:	learn: 283.7822843	total: 21.7s	remaining: 6.7s
382:	learn: 283.6900748	total: 21.7s	remaining: 6.64s
383:	learn: 283.5767969	total: 21.8s	remaining: 6.58s
384:	learn: 283.5169153	total: 21.8s	remaining: 6.53s
385:	learn: 283.4496524	total: 21.9s	remaining: 6.47s
386:	learn: 283.3673564	total: 22s	remaining: 6.41s
387:	learn: 283.3089439	total: 22s	remaining: 6.35s
388:	learn: 283.2241155	total: 22.1s	remaining: 6.29s
389:	learn: 283.1484096	total: 22.1s	remaining: 6.24s
390:	learn: 283.0624061	total: 22

CatboostError: (The process cannot access the file because it is being used by another process.) c:/goagent/pipelines/buildmaster/catboost.git/util/system/file.cpp:621: can't open "catboost_info\\meta.tsv" with mode CreateAlways|WrOnly|Seq (0x00000034)

In [ ]:
grid_search.best_params_

In [ ]:
y_pred_cat=cb_model.predict(X_test)
#y_pred_ran=np.absolute(y_pred_ran)

In [52]:
from sklearn.metrics import mean_squared_log_error
cat_error=mean_squared_log_error(y_test,y_pred_cat,multioutput='raw_values')
final_cat_error=100*cat_error
final_cat_error

array([27.86893231])

In [54]:
y_pred_cat=cb_model.predict(test_processed)

In [56]:
df_submission=pd.DataFrame({'id':test.index,'num_orders':y_pred_cat})
df_submission.to_csv('cat_reg3.csv',index=False)